In [7]:
from selenium import webdriver
# from selenium.webdriver.common.keys import Keys
# import time
from bs4 import BeautifulSoup

In [21]:
driver = webdriver.Chrome('./mac_m1/chromedriver')
driver.get('https://edit.tosdr.org/users/sign_in')
driver.implicitly_wait(2)
driver.find_element_by_id('user_email').send_keys('shparksue@gmail.com')
driver.find_element_by_id('user_password').send_keys('2022-2NLPproject')
driver.find_element_by_xpath('//*[@id="new_user"]/div[2]/input').click()

driver.get('https://edit.tosdr.org/services')
driver.implicitly_wait(3)

In [44]:
services_html = driver.page_source
services_soup = BeautifulSoup(services_html, 'html.parser')
table = services_soup.select_one('table.table.table-striped')

In [86]:
driver.get("https://edit.tosdr.org//services/225/annotate")
soup = BeautifulSoup(driver.page_source, 'html.parser')
docs = soup.select('div.panel.panel-default')

In [110]:
doc = docs[3]
title = doc.select_one('h3').text
original_text = doc.select_one('.panel-body.documentContent')['data-content']
text = doc.select_one('.panel-body.documentContent > div')
summary = doc.select_one('.panel-body.documentContent > p > a')

print(title)
print(text)
print(summary)

Terms and Conditions of Use
<div data-document-id="67" data-offset="0" data-revision="latest"> </div>
<a data-document-id="67" data-offset="1" href="/points/12719" title="There is a date of the last update of the terms">Spotify Terms and Conditions of Use <span class="hiddenTag">&lt;p&gt;</span>Effective as of February 7, 2019</a>


In [88]:
import re

out = re.split('<.+?>', text.text)
print(out)
res = []
for segments in out:
    res.extend(s.strip() for s in segments.split('\n') if s.strip())
print(res)

[' Voice on Spotify® ', 'Spotify can be controlled with your voice.\nBy activating voice controls, you enable a variety of features, like finding songs and playlists using your voice and even interacting with some advertisements.\nYour privacy and security comes first.\nLearn more below about how Spotify collects and processes your voice data.', ' What voice data does Spotify collect? ', 'When you use voice features, your voice input and other information will be sent to Spotify.\nThis includes audio recordings and transcripts of what you say, and other related information such as the content that was returned to you by Spotify.', ' ']
['Voice on Spotify®', 'Spotify can be controlled with your voice.', 'By activating voice controls, you enable a variety of features, like finding songs and playlists using your voice and even interacting with some advertisements.', 'Your privacy and security comes first.', 'Learn more below about how Spotify collects and processes your voice data.', 'Wha

In [89]:
def parse_section(text):
    sentences = []
    for segments in re.split('<.+?>', text):
        sentences.extend(segment.strip() for segment in segments.split('\n') if segment.strip())
    return sentences

In [111]:
ptr = doc.select_one('.panel-body.documentContent > p')  # separator just before the document title
full_doc = []
summary = []
for section in ptr.next_siblings:
    if section.text.strip():  # tag content is not empty
        sentences = parse_section(section.text)
        if section.select('a'):  # is hyperlinked, i.e., is deemed important by the users
            summary.extend(sentences)
        full_doc.extend(sentences)
print(full_doc)
print(summary)

['Spotify Terms and Conditions of Use', 'Effective as of February 7, 2019', '1 Introduction', '2 Changes to the Agreements', '3 Enjoying Spotify', '4 Payments and cancellations', '5 Using our service', '6 Third Party Applications and Devices', '7 User-Generated Content', '8 Rights you grant us', '9 User guidelines', '10 Infringement and reporting User Content', '11 Service limitations and modifications', '12 Brand Accounts', '13 Spotify Support Community', '14 Customer support', '15 Export control', '16 Term and termination', '17 Warranty disclaimer', '18 Limitation and time for filing', '19 Third party rights', '20 Entire agreement', '21 Severability, waiver and interpretation', '22 Assignment', '23 Indemnification', '24 Choice of law, mandatory arbitration, and venue', '25 Contact us', 'Hello, and welcome to the Spotify Terms and Conditions of Use (“Terms”).', 'The Terms you see below are important because they:', 'Outline your legal rights on Spotify', 'Explain the rights you give t

In [ ]:
def parse_section(text):
    sentences = []
    for segments in re.split('<.+?>', text):
        sentences.extend(segment.strip() for segment in segments.split('\n') if segment.strip())
    return sentences

def generate_document_data(document_elements):
    full_doc = []
    summary = []
    ptr = doc.select_one('.panel-body.documentContent > p')  # separator just before the document title
    for section in ptr.next_siblings:
        if section.text.strip():  # tag content is not empty
            sentences = parse_section(section.text)
            if section.select('a'):  # is hyperlinked, i.e., is deemed important by the users
                summary.extend(sentences)
            full_doc.extend(sentences)
    return full_doc, summary

def parse_document(document_data, document_elements):
    title = document_elements.select_one('h3').text
    original_text, summary = generate_document_data(document_elements)
    return document_data | {'document_title': title,
            'original_text_length': len(original_text),
            'summary_length': len(summary),
            'original_text': original_text, 
            'summary': summary}  # merge dictionary

def parse_service(service_name, documents_html):
    documents_soup = BeautifulSoup(documents_html, 'html.parser')
    for document_elements in documents_soup.select('div.panel.panel-default'):
        document_data = {'service': service_name}
        document_data = parse_document(document_data, document_elements)
    
        

In [31]:
BASE_URL = 'https://edit.tosdr.org/'
for idx, row in enumerate(table.find_all('tr', {'data-classification': ['A', 'B', 'C', 'D', 'E']})):
    columns = row.find_all('td')
    service = columns[1].text.strip()
    url = BASE_URL + columns[4].find('a', href=True)['href']
    print(service)
    print(url)
    documents_html = driver.get(url).page_source
    parse_service(documents_html)
    if idx == 1:
        break

AttributeError: ResultSet object has no attribute 'find_all'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [112]:
from tqdm import tqdm
import json

class ToSDRScraper():
    def __init__(self, os='mac_m1'):
        self.driver = webdriver.Chrome(f'./{os}/chromedriver')
        self.out_file = "./tosdr.jsonl"
        self.write_f = open(self.out_file, 'w')
        self.base_url = 'https://edit.tosdr.org'
        
        
    def login(self, email='shparksue@gmail.com', password='2022-2NLPproject'):
        """Activates authenticated session"""
        self.driver.get('https://edit.tosdr.org/users/sign_in')
        self.driver.implicitly_wait(2)
        
        self.driver.find_element_by_id('user_email').send_keys(email)
        self.driver.find_element_by_id('user_password').send_keys(password)
        self.driver.find_element_by_xpath('//*[@id="new_user"]/div[2]/input').click()
    
    
    def get_html_source(self, url: str):
        """Get the HTML source to directly instantiate a new BeautifulSoup object (possibly for debugging purposes)"""
        self.driver.get(url)
        self.driver.implicitly_wait(2)
        return self.driver.page_source
    
    
    def scrape_services(self):
        """Get urls for the annotated documents of each service and process each document"""
        self.driver.get('https://edit.tosdr.org/services')
        self.driver.implicitly_wait(4)  # takes some time to load the full page
        
        services_html = self.driver.page_source
        services_soup = BeautifulSoup(services_html, 'html.parser')
        table = services_soup.select_one('table.table.table-striped')
        for row in tqdm(table.find_all('tr', {'data-classification': ['A', 'B', 'C', 'D', 'E']})):
            columns = row.find_all('td')
            service = columns[1].text.strip()
            url = self.base_url + columns[4].find('a', href=True)['href']
            documents_html = self.driver.get(url).page_source
            self.scrape_documents_per_service(service, url, documents_html)
            
            
    def scrape_documents_per_service(self, service_name, url, documents_html, out_file_name=None):
        """Scrape each ToS document of each service and write the data into .jsonl format"""
        if out_file_name:  # for DEBUG
            self.out_file = out_file_name
            self.write_f = open(self.out_file, 'w')
            
        documents_soup = BeautifulSoup(documents_html, 'html.parser')
        for document_elements in documents_soup.select('div.panel.panel-default'):
            document_data = {'service': service_name, 'url': url}
            document_data = self._parse_document(document_data, document_elements)
            if document_data == None:
                continue
            self.write_f.write(json.dumps(document_data) + '\n')
            
    
    def _parse_document(self, document_data, document_elements):
        """Parse each document return a dictionary of the structured information"""
        title = document_elements.select_one('h3').text
        original_text, summary = self._generate_document_data(document_elements)
        if summary == None:
            return None
        
        return document_data | {'document_title': title,
                'original_text_length': len(original_text),
                'summary_length': len(summary),
                'original_text': original_text, 
                'summary': summary}  # merge dictionary
        
        
    def _generate_document_data(self, document_elements):
        """Iterate through the sections and divide the parsed sections of the documents into the original text and summary"""
        # NOTE: the first or last sentence in the annotated section, and the sentences before and after it can contain incomplete phrases
        full_doc = []
        summary = []
        ptr = document_elements.select_one('.panel-body.documentContent > p')
        for section in ptr.next_siblings:  # iterate through the contents
            if section.text.strip():  # tag content is not empty
                sentences = self._parse_section(section.text)
                if section.select('a'):  # is hyperlinked, i.e., annotated by users
                    summary.extend(sentences)
                full_doc.extend(sentences)
        if len(summary) == 0:
            return full_doc, None
        return full_doc, summary
    
    
    def _parse_section(self, text):
        """Break down each section into sentences"""
        sentences = []
        for segments in re.split('<.+?>', text):
            sentences.extend(segment.strip() for segment in segments.split('\n') if segment.strip())
        return sentences

In [116]:
scraper = ToSDRScraper()

In [117]:
scraper.login()

In [118]:
spotify_url = 'https://edit.tosdr.org//services/225/annotate'
spotify_html = scraper.get_html_source(spotify_url)
scraper.scrape_documents_per_service('Spotify', documents_html=spotify_html, out_file_name="./tosdr_spotify.jsonl")